In [46]:
def gcd(a, b):
    if a == 0:
        return b, 0, 1
    if b == 0:
        return a, 1, 0
    else:
        r_0, r_1 = a, b 
        u_0, u_1 = 1, 0
        v_0, v_1 = 0, 1

        while r_1 != 0:
            q = r_0 // r_1
            r_0, r_1 = r_1, r_0 - q*r_1
            u_0, u_1 = u_1, u_0 - q*u_1
            v_0, v_1 = v_1, v_0 - q*v_1

    return r_0, u_0, v_0

def inv(a, b):
    return gcd(a, b)[1]

def mod_comp(a, b, n):
    d = gcd(a, n)[0]
    s = []
    if d == 1:
        return (inv(a, n) * b) % n
    elif b % d != 0: 
        return "No solutions."
    else:
        a = a / d
        b = b / d
        n = n / d
        x_0 = (inv(a, n) * b) % n
        for i in range(0, d):
            s.append(x_0 + i * n)
        return s
